In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
 dbutils.widgets.text("load_flags","0")

In [0]:
load_flags =int(dbutils.widgets.get("load_flags"))

### data reading from source


In [0]:
df= spark.sql("select * from databricks_ete.silver.customers_silver")

In [0]:
df.dropDuplicates(['customer_id'])


***dividing new vs old records***

In [0]:
if load_flags==0:
    df_old= spark.sql("select dimkeys, customer_id,createdate,update_date from databricks_ete.gold.dimcustomers")
    
else :
    df_old= spark.sql("select 0 as dimkeys, 0 customer_id,0 create_date,0 update_date from databricks_ete.silver.customers_silver where 1=0")

In [0]:
df_old.display()

renaming old df coulmn names

In [0]:
df_old=df_old.withColumnRenamed("dimkeys","old_dimkeys")\
.withColumnRenamed("customer_id","old_customer_id")\
.withColumnRenamed("create_date","old_create_date")\
.withColumnRenamed("update_date","old_update_date")


***applying join on old records***

In [0]:
df_join=df.join(df_old , df['customer_id']==df_old['old_customer_id'],'left')

In [0]:
df_join.display()

seperating old vs new records

In [0]:
df_new = df_join.filter(df_join['old_dimkeys'].isNull())

In [0]:
df_old = df_join.filter(df_join['old_dimkeys'].isNotNull())

preparing df old

In [0]:
df_old= df_old.drop('old_customer_id','old_update_date')

# Renaming "old_DimCustomerKey" to "DimCustomerKey"

df_old = df_old.withColumnRenamed("old_dimkeys", "DimKeys")
#renaming old create date
df_old= df_old.withColumnRenamed('old_create_date','createdate')
df_old= df_old.withColumn('createdate',to_timestamp(col('createdate')))
#creating old update date
df_old= df_old.withColumn('update_date',current_timestamp())


In [0]:
df_old.display()

preparing df_new 


In [0]:
df_new= df_new.drop('old_dimkeys','old_customer_id','old_create_date','old_update_date')
#creating old update date
df_new= df_new.withColumn('update_date',current_timestamp())
df_new= df_new.withColumn('createdate',current_timestamp())

In [0]:
df_new.display()


surrogate keys - from 1

In [0]:
df_new=df_new.withColumn("dimkeys", monotonically_increasing_id()+lit(1))

In [0]:
df_new.limit(10).display()


adding max surrogate key

In [0]:
if load_flags==1:
  max_surrogate_key=0
else :
  df_maxsur= spark.sql("select max(dimkeys) as max_surrogate_key from  databricks_ete.gold.dimcustomers")
  #converting max sur key into variable
  max_surrogate_key = df_maxsur.collect()[0]['max_surrogate_key']

In [0]:
df_new= df_new.withColumn("dimkeys", lit(max_surrogate_key)+col("dimkeys"))

union of df_old and df_new

In [0]:
df_final= df_new.unionByName(df_old)


In [0]:
df_final.display()

scd type 1


In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_ete.gold.dimcustomers"):
  dlt_table = DeltaTable.forPath(spark, "abfss://gold@eteproject.dfs.core.windows.net/dimcustomers")
  dlt_table.alias("tgt").merge(df_final.alias("src"), "tgt.dimkeys=src.dimkeys").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
  
else:
  df_final.write.mode("overwrite").option("path","abfss://gold@eteproject.dfs.core.windows.net/dimcustomers").saveAsTable("databricks_ete.gold.dimcustomers")